# Домашнее задание к лекции "Основы веб-скрапинга"


### Обязательная часть

Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

> <дата> - <заголовок> - <ссылка на материал>

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.


In [1]:
#подключаем библиотеки
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
import re

In [2]:
# Vers.01 -- date, title, link
def get_habr_posts(query):
    habr_blog = pd.DataFrame()
    URL = 'https://habr.com/ru/search/'
    params = {
        'q': query
    }
    req = requests.get(URL, params=params)
    time.sleep(0.3)
    soup = BeautifulSoup(req.text)
    articles = soup.find_all('article', class_='tm-articles-list__item')
    for article in articles:
        title = article.find('h2', class_='tm-article-snippet__title').text
        link = 'https://habr.com' + article.find('h2', class_='tm-article-snippet__title').find('a').get('href')
        date = article.find('span', class_='tm-article-snippet__datetime-published').text.strip()
        row = {'date': date, 'title': title, 'link': link}
        habr_blog = pd.concat([habr_blog, pd.DataFrame([row])]) 
    return habr_blog.reset_index(drop=True)

res = get_habr_posts(['python', 'анализ данных'])
res

,date,title,link
0,10 декабря 2021 в 13:15,Интенсив для повышения квалификации: как испол...,https://habr.com/ru/company/netologyru/news/t/...
1,27 июня 2017 в 13:51,Использование Python и Excel для обработки и а...,https://habr.com/ru/company/otus/blog/331746/
2,30 июня 2017 в 10:52,Использование Python и Excel для обработки и а...,https://habr.com/ru/company/otus/blog/331998/
3,9 июня в 21:30,Шаблон новичка на пути PANDAS в искусстве анал...,https://habr.com/ru/post/670668/
4,6 мая 2021 в 06:01,"Python, исследование данных и выборы: часть 3",https://habr.com/ru/post/556042/
5,22 сентября 2020 в 16:43,Напишем и поймем Decision Tree на Python с нул...,https://habr.com/ru/post/520204/
6,18 октября 2021 в 15:08,AI Journey Contest 2021: какие задачи мы подго...,https://habr.com/ru/company/sbercloud/news/t/5...
7,30 июня 2014 в 23:06,Обзор наиболее интересных материалов по анализ...,https://habr.com/ru/post/228187/
8,23 ноября 2014 в 16:49,Обзор наиболее интересных материалов по анализ...,https://habr.com/ru/post/243967/
9,18 января 2015 в 18:02,Обзор наиболее интересных материалов по анализ...,https://habr.com/ru/post/248165/


### Дополнительная часть (необязательная)

Функция из обязательной части задания должна быть расширена следующим образом:

* кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
* в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:

> <дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>

In [3]:
# Функция для замены нескольких значений
def multiple_replace(target_str, replace_values):
    # получаем заменяемое: подставляемое из словаря в цикле
    for i, j in replace_values.items():
        # меняем все target_str на подставляемое
        target_str = target_str.replace(i, j)
    return target_str

# создаем словарь со значениями и строку, которую будет изменять
replace_values = {'\n': ' ', '\r' : ' '}

In [6]:
# Функция для добавления полного текста статьи
def add_full_text(posts_df):
    i = 0
    for el in posts_df['link']:
        req = requests.get(el).text
        soup = BeautifulSoup(req)
        time.sleep(0.3)
        post_text = soup.find('div', class_=['article-formatted-body article-formatted-body article-formatted-body_version-1', 'article-formatted-body article-formatted-body article-formatted-body_version-2']).text.strip()
        post_text = multiple_replace(post_text, replace_values)
        posts_df.loc[i, 'text'] = post_text
        i += 1
    return posts_df

In [7]:
# Vers.02 -- add full_text, likes and page
def get_habr_posts_full(page, query):
    habr_blog = pd.DataFrame()
    URL = 'https://habr.com/ru/search/page' + str(page) + '/'
    params = {
        'q': query
    }
    req = requests.get(URL, params=params)
    time.sleep(0.3)
    soup = BeautifulSoup(req.text)
    articles = soup.find_all('article', class_='tm-articles-list__item')
    for article in articles:
        title = article.find('h2', class_='tm-article-snippet__title').text
        link = 'https://habr.com' + article.find('h2', class_='tm-article-snippet__title').find('a').get('href')
        date = article.find('span', class_='tm-article-snippet__datetime-published').text.strip()
        span_likes = article.find_all('span', class_=['tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_positive tm-votes-meter__value_appearance-article tm-votes-meter__value_rating', 'tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_appearance-article tm-votes-meter__value_rating'])
        regex = r'Всего голосов.*?(\d+)'
        likes = re.findall(regex, str(span_likes))
        likes = int(likes[0])
        row = {'date': date, 'title': title, 'link': link, 'likes': likes}
        habr_blog = pd.concat([habr_blog, pd.DataFrame([row])]) 
    
    res = habr_blog.reset_index(drop=True)
    return  add_full_text(res)

get_habr_posts_full(2, ['python', 'анализ данных'])

,date,title,link,likes,text
0,20 ноября 2013 в 03:02,"Профилирование и отладка Python, инструменты",https://habr.com/ru/company/vk/blog/202832/,72,В предыдущей статье мы на практике разобрались...
1,12 мая 2020 в 09:12,"Повторяем когортный анализ, выполненный в Powe...",https://habr.com/ru/post/501492/,5,Добрый день уважаемые читатели! Поводом для на...
2,8 сентября 2017 в 16:59,Использование Python для обработки в реальном ...,https://habr.com/ru/post/337494/,15,Постановка задачи Цифровые и аналоговые датчи...
3,11 октября 2013 в 17:58,Введение в визуализацию данных при анализе с п...,https://habr.com/ru/post/197212/,20,"Доброго времени суток, уважаемые читатели. Ка..."
4,8 апреля 2018 в 16:31,Анализ данных с использованием Python,https://habr.com/ru/post/353050/,35,Язык программирования Python в последнее время...
5,29 апреля 2020 в 12:39,Практическое руководство по разработке бэкенд-...,https://habr.com/ru/company/yandex/blog/499534/,70,"Привет, меня зовут Александр Васин, я бэкенд-р..."
6,7 апреля 2021 в 18:51,Как сделать полнотекстовую поисковую машину на...,https://habr.com/ru/company/skillfactory/blog/...,8,Полнотекстовый поиск — неотъемлемая часть наше...
7,5 августа 2021 в 08:18,Спектральный анализ временных рядов с помощью ...,https://habr.com/ru/post/571344/,3,С развитием информационных технологий професси...
8,3 сентября 2019 в 10:17,Полезная help-ссылка для работы с данными,https://habr.com/ru/company/vk/blog/465853/,48,"Хабр, привет. Представляю вам главную help-ссы..."
9,7 октября 2013 в 14:12,Введение в анализ сложности алгоритмов (часть 1),https://habr.com/ru/post/196560/,106,От переводчика: данный текст даётся с незначит...
